In [1]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.3MB/s 
     |████████████████████████████████| 8.2MB 4.8MB/s 


In [2]:
import pymorphy2
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import json

morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def collect_text(url):
  """Downloads html from given url, gets text of letter and source,
    clears everything else, split text into sentences.
  """
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")

  for script in soup(["script", "style"]):
    script.extract()
  text = soup.get_text()

  lines = (line.strip() for line in text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  def is_date(o):
    x = o.split()
    if len(x) < 2:
      return False
    x = x[-2]
    if(len(x) == 4 and x[0] == '1' and x[1] == '9'):
      return True
    return False

  source = ''

  kill = False

  def TCHEKHOV(chunk):
    if 'ЧЕХОВ' in chunk:
      return True
    return False
  
  tch = ''

  for chunk in (chunks):
    if TCHEKHOV(chunk):
      tch = chunk
    if kill and chunk:
      source += chunk
      chunk = None
      break
    if is_date(chunk):
      source += (chunk)
      source += ' '
      chunk = None
      kill = True
    else:
      chunk = None


  erase = False

  def eraser(chunk):
    nonlocal erase
    if chunk == 'ПОИСК:':
      erase = True
    if erase:
      return False
    if not chunk:
      return False
    return True

  text = '\n'.join(chunk for chunk in chunks if eraser(chunk))

  sentences = nltk.tokenize.sent_tokenize(text)
  source = "Письмо " + tch + ' ' + source
  return source, sentences

In [4]:
def annotate_text(source, sentences):
  """ Parses given sentences using pymorphy and creates corpora. """
  end_s = []
  for s in sentences:
    original = s
    original = original.lower()
    p = ',:@.-()/!'
    original_cleaned = original
    for i in p:
      original_cleaned = original_cleaned.replace(i, '')
    original_splitted = [ i for i in re.split(r'\W+',original) if i]
    
    lemmatized = []
    for j in original_splitted:
        lemmatized.append(morph.parse(j)[0].normal_form)
    tags = []
    for word in original_splitted:   
        tag = morph.parse(word)[0].tag
        tags.append(str(tag))
    end_s.append((source, s, original_splitted, lemmatized, tags))
  return end_s

In [10]:
def main():
  """Creates file.txt with information about each sentence
    in each letter downloaded from given url if it is suitable.
    """
  corpora = []
  total_quantity_of_words = 0
  for i in range(24, 354):
    if i == 193 or i == 194 or i == 202 or i == 203 or i == 204 or i == 205 or i == 313 or i == 326:
      continue
    if i < 100:
      url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st0' + str(i) + '.shtml'
    else:
      url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st' + str(i) + '.shtml'
    source, sentences = collect_text(url)
    annotated_text = annotate_text(source, sentences)
    for s in annotated_text:
      total_quantity_of_words += len(s[3])
    corpora += (annotated_text)
  
  data = corpora

  with open('corporaora.txt', 'w') as outfile:
      json.dump(data, outfile)

  with open('corporaora.txt') as json_file:
      data = json.load(json_file)
      print(data[:10], total_quantity_of_words)

In [11]:
if __name__ == '__main__':
    main()

[['Письмо 2*. О. Л. КНИППЕР-А. П. ЧЕХОВУ 27 ноября 1902 г. Москва', 'Голубчик мой, дуся моя, опять ты уехал... Я одна, сижу в спальной и строчу.', ['голубчик', 'мой', 'дуся', 'моя', 'опять', 'ты', 'уехал', 'я', 'одна', 'сижу', 'в', 'спальной', 'и', 'строчу'], ['голубчик', 'мой', 'дуся', 'мой', 'опять', 'ты', 'уехать', 'я', 'один', 'сидеть', 'в', 'спальная', 'и', 'строчить'], ['NOUN,anim,masc sing,nomn', 'ADJF,Apro masc,sing,nomn', 'NOUN,anim,femn,Name sing,nomn', 'ADJF,Apro femn,sing,nomn', 'ADVB', 'NPRO,2per sing,nomn', 'VERB,perf,intr masc,sing,past,indc', 'NPRO,1per sing,nomn', 'ADJF,Apro,Anum femn,sing,nomn', 'VERB,impf,intr sing,1per,pres,indc', 'PREP', 'NOUN,inan,femn sing,gent', 'CONJ', 'VERB,impf,tran sing,1per,pres,indc']], ['Письмо 2*. О. Л. КНИППЕР-А. П. ЧЕХОВУ 27 ноября 1902 г. Москва', 'Все тихо.', ['все', 'тихо'], ['всё', 'тихо'], ['PRCL', 'ADVB']], ['Письмо 2*. О. Л. КНИППЕР-А. П. ЧЕХОВУ 27 ноября 1902 г. Москва', 'Ты, верно, около Орла, или уже в нем.', ['ты', 'верно', 